In [1]:
import numpy as np
import pandas as pd

## Combining and Merging Datasets

`pandas.merge`
Connect rows in DataFrames based on one or more keys. This will be familiar
to users of SQL or other relational databases, as it implements database join
operations.
`pandas.concat`
Concatenate or “stack” objects together along an axis.
`combine_first`
Splice together overlapping data to fill in missing values in one object with values
from another.

### Database-Style DataFrame Joins

In [2]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
"data1": pd.Series(range(7), dtype="Int64")})

In [3]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
"data2": pd.Series(range(3), dtype="Int64")})

In [4]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [5]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a many-to-one join; the data in df1 has multiple rows labeled
a and b, whereas df2 has only one row for each value in the key column. Calling
pandas.merge with these objects, we obtain:

In [8]:
df1.merge(df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [9]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [10]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
"data1": pd.Series(range(7), dtype="Int64")})

In [11]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
"data2": pd.Series(range(3), dtype="Int64")})

In [13]:
print(df3)
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


In [12]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


By default, pandas.merge does an "inner" join; the keys in the result are
the intersection, or the common set found in both tables. Other possible options are
"left", "right", and "outer". The outer join takes the union of the keys, combining
the effect of applying both left and right joins:

In [14]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [15]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


![merge](Assets\merge_on.png)

In [16]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
"data1": pd.Series(range(6), dtype="Int64")})

In [17]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
"data2": pd.Series(range(5), dtype="Int64")})

In [18]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [19]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [20]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [21]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [22]:
 left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
"key2": ["one", "two", "one"],
"lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
"key2": ["one", "one", "one", "two"],
"rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

In [23]:
print(left)
print(right)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [24]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [25]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [26]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


![mergemethod](Assets\mergemethod1.png)
![mergemethod](Assets\mergemethod2.png)

## Merging on Index

In [27]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
"value": pd.Series(range(6), dtype="Int64")})

In [28]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [29]:
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [30]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [31]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


DataFrame has a join instance method to simplify merging by index. It can also be
used to combine many DataFrame objects having the same or similar indexes but
nonoverlapping columns. In the prior example, we could have written:


In [32]:
left1.join(right1, how='outer')

,key,value,group_val
0,a,0,NaN
1,b,1,NaN
2,a,2,NaN
3,a,3,NaN
4,b,4,NaN
5,c,5,NaN
a,NaN,<NA>,3.5
b,NaN,<NA>,7.0


In [33]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


## Concatenating Along an Axis

In [34]:
arr = np.arange(12).reshape((3, 4))

In [35]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [36]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [37]:
print(s1)
print(s2)
print(s3)

a    0
b    1
dtype: Int64
c    2
d    3
e    4
dtype: Int64
f    5
g    6
dtype: Int64


In [38]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [39]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [44]:
s1

a    0
b    1
dtype: Int64

In [42]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [41]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [43]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1
